In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
data = pd.read_csv('creditcard.csv')

# Display the first few rows of the dataframe
print(data.head())

# Check for any missing values
print(data.isnull().sum())

# Assuming no missing values, proceed with scaling
scaler = StandardScaler()
data['Amount'] = scaler.fit_transform(data['Amount'].values.reshape(-1, 1))

# Separate features and labels
X = data.drop('class', axis=1)
y = data['class']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Define the Transformer-based Autoencoder
class TransformerAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super(TransformerAutoencoder, self).__init__()
        self.input_adapter = nn.Linear(input_dim, 128)
        self.encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=128, nhead=8), num_layers=6)
        self.decoder = nn.Linear(128, input_dim)
        
    def forward(self, x):
        x = self.input_adapter(x)
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the TransformerAutoencoder
input_dim = X_train.shape[1]
autoencoder = TransformerAutoencoder(input_dim)

# Define the contrastive loss
contrastive_loss = nn.CosineEmbeddingLoss()

# Initialize the optimizer
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Training Loop
epochs = 10
for epoch in range(epochs):
    optimizer.zero_grad()
    output = autoencoder(X_train_tensor)
    loss = contrastive_loss(output, X_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Generate fake data using GAN
# Define Generator and Discriminator models

# Define the Generator
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.fc = nn.Linear(input_dim, 64)
        self.out = nn.Linear(64, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc(x))
        x = torch.sigmoid(self.out(x))
        return x

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Initialize the Generator and Discriminator
generator = Generator(input_dim, input_dim)
discriminator = Discriminator(input_dim)

# Initialize optimizers for Generator and Discriminator
g_optimizer = optim.Adam(generator.parameters(), lr=0.001)
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)

# Training Loop for GAN
for epoch in range(epochs):
    # Train Discriminator
    real_data = X_train_tensor
    fake_data = generator(X_train_tensor)
    
    d_optimizer.zero_grad()
    
    real_pred = discriminator(real_data)
    fake_pred = discriminator(fake_data.detach())
    d_loss = -torch.log(real_pred).mean() - torch.log(1 - fake_pred).mean()
    
    d_loss.backward()
    d_optimizer.step()
    
    # Train Generator
    g_optimizer.zero_grad()
    
    fake_pred = discriminator(fake_data)
    g_loss = -torch.log(fake_pred).mean()
    
    g_loss.backward()
    g_optimizer.step()
    
    print(f'Epoch {epoch+1}, Generator Loss: {g_loss.item()}, Discriminator Loss: {d_loss.item()}')

fake_data = generator(X_test_tensor).detach().numpy()

# Evaluate the model
# Assuming anomaly detection based on thresholding

# Visualizing the results
sns.countplot(np.where(y_pred == 0, "Normal", "Anomaly"))
plt.title('Counts of Normal vs Anomaly Predictions')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()
